In [0]:
%sql

with sor as (
  select 
    country, cast(invoicedate as date) invoicedate
    , price, quantity
    , invoice
  from lab_2026.silver_online_retail
)
select 
  country, invoicedate 
  , sum(price * quantity) total_revenue
  , count(distinct invoice) total_orders
from sor
group by 1,2
LIMIT 10

country,invoicedate,total_revenue,total_orders
United Kingdom,2009-12-01,46303.85,154
Portugal,2009-12-01,2454.68,1
United Kingdom,2009-12-02,55371.47,125
United Kingdom,2009-12-03,67960.25,144
United Kingdom,2009-12-04,37380.85,100
United Kingdom,2009-12-06,22362.36,87
United Kingdom,2009-12-07,38942.09,110
Greece,2009-12-07,610.95,1
United Kingdom,2009-12-08,40942.34,157
United Kingdom,2009-12-09,37786.78,108


In [0]:
%sql

DROP TABLE IF EXISTS lab_2026.gold_daily_sales;

CREATE TABLE IF NOT EXISTS lab_2026.gold_daily_sales(
  country string, 
  invoice_date date, 
  total_revenue decimal(38,2),
  total_orders long,
  _insert_timestamp timestamp,
  _update_timestamp timestamp
);

In [0]:
%sql

DESCRIBE EXTENDED lab_2026.gold_daily_sales;

col_name,data_type,comment
country,string,null
invoice_date,date,null
total_revenue,"decimal(38,2)",null
total_orders,bigint,null
_insert_timestamp,timestamp,null
_update_timestamp,timestamp,null
,,
# Detailed Table Information,,
Catalog,workspace,
Database,lab_2026,


In [0]:
from pyspark.sql import functions as F, types as T

online_retail_df = (
    spark.read.table('lab_2026.silver_online_retail')
    .withColumn('invoice_date', F.col('invoicedate').cast(T.DateType()))
    .groupBy('country', 'invoice_date')
    .agg(
        F.sum(F.col('price') * F.col('quantity')).alias('total_revenue'),
        F.countDistinct(F.col('invoice')).alias('total_orders')
    )
    .withColumn('_insert_timestamp', F.current_timestamp())
    .withColumn('_update_timestamp', F.current_timestamp())
)

online_retail_df.limit(5).display()

country,invoice_date,total_revenue,total_orders,_insert_timestamp,_update_timestamp
United Kingdom,2009-12-01,46303.85,154,2026-01-02T12:14:23.402Z,2026-01-02T12:14:23.402Z
Portugal,2009-12-01,2454.68,1,2026-01-02T12:14:23.402Z,2026-01-02T12:14:23.402Z
United Kingdom,2009-12-02,55371.47,125,2026-01-02T12:14:23.402Z,2026-01-02T12:14:23.402Z
United Kingdom,2009-12-03,67960.25,144,2026-01-02T12:14:23.402Z,2026-01-02T12:14:23.402Z
United Kingdom,2009-12-04,37380.85,100,2026-01-02T12:14:23.402Z,2026-01-02T12:14:23.402Z


In [0]:
from delta.tables import DeltaTable

(
    DeltaTable
    .forName(spark, 'lab_2026.gold_daily_sales')
    .alias('gold')
    .merge(
        online_retail_df.alias('src')
        , 'gold.country = src.country AND gold.invoice_date = src.invoice_date'
    )
    .whenMatchedUpdate(
        set = {
            'total_revenue': 'src.total_revenue'
            , 'total_orders': 'src.total_orders'
            , '_update_timestamp': 'src._update_timestamp'
        }
    )
    .whenNotMatchedInsertAll()
    .execute()
)

DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

In [0]:
%sql

select * from lab_2026.gold_daily_sales limit 10;

country,invoice_date,total_revenue,total_orders,_insert_timestamp,_update_timestamp
United Kingdom,2011-10-30,29745.96,91,2026-01-02T12:16:48.698Z,2026-01-02T12:16:48.698Z
Belgium,2011-10-30,299.06,1,2026-01-02T12:16:48.698Z,2026-01-02T12:16:48.698Z
Spain,2011-10-30,2176.45,1,2026-01-02T12:16:48.698Z,2026-01-02T12:16:48.698Z
Sweden,2011-10-30,454.20,1,2026-01-02T12:16:48.698Z,2026-01-02T12:16:48.698Z
Italy,2011-10-30,343.00,1,2026-01-02T12:16:48.698Z,2026-01-02T12:16:48.698Z
France,2011-10-30,214.74,1,2026-01-02T12:16:48.698Z,2026-01-02T12:16:48.698Z
EIRE,2011-10-30,501.66,1,2026-01-02T12:16:48.698Z,2026-01-02T12:16:48.698Z
United Kingdom,2010-06-07,42378.32,112,2026-01-02T12:16:48.698Z,2026-01-02T12:16:48.698Z
Germany,2010-06-07,2009.93,6,2026-01-02T12:16:48.698Z,2026-01-02T12:16:48.698Z
Austria,2010-06-07,259.85,1,2026-01-02T12:16:48.698Z,2026-01-02T12:16:48.698Z
